<img src='https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcQ-VfNtOyJbsaxu43Kztf_cv1mgBG6ZIQZEVw&usqp=CAU'>

# Procesamiento de Lenguaje Natural

## Taller #11: tweet 

Guillermo Andres Cardona
cod:616202023

In [110]:

import pandas as pd
import tweepy
import regex
import emoji

import plotly.io as pio
import plotly.express as px
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

import matplotlib.pyplot as plt
%matplotlib inline
init_notebook_mode(connected=True)


In [111]:
# Leer las llaves
keys = pd.read_csv("G:/token/tweet.csv", header=None)
keys = dict(zip(keys[0],keys[1]))
keys.keys()

# Autenticar a través de la API 
auth = tweepy.OAuthHandler(keys['api_key'], keys['api_secret_key'])
auth.set_access_token(keys['access_token'], keys['access_token_secret'])
api = tweepy.API(auth, wait_on_rate_limit=True)



In [112]:

  #api.update_status('Colombia 🥺🥺🥺')


In [113]:
tweets = tweepy.Cursor(api.search,
                       q=['colombia -RT'],#RT quitar los retwteados
                       lang='es',
                       #geocode='4.7110,-74.0721,100km',
                       tweet_mode='extended',
                       include_rts=False,
                       ).items(100)


# Objeto generador
t = [tweet for tweet in tweets]


In [114]:
t[1].full_text
t[1].created_at
t[1].user
t[1].user.location

'Bogotá,D.C.,Colombia'

In [115]:
t_processed = []

for tweet in t: 
    temp = []
    temp.append(tweet.full_text)
    temp.append(tweet.created_at)
    temp.append('Bogotá')
    temp.append(tweet.user.location)
    temp.append(tweet.id)
    temp.append(f"https://twitter.com/i/web/stuatus/{tweet.id}")
    
    t_processed.append(temp)
    
    
data = pd.DataFrame(t_processed)
data.columns = ['tweets', 'date', 'city', 'user_location', 'id', 'link']
data.head()


,tweets,date,city,user_location,id,link
0,#ATENCION | Urgente\n@ArchivaColombia @WOLA_or...,2021-05-19 23:51:08,Bogotá,Holanda,1395165156779479040,https://twitter.com/i/web/stuatus/139516515677...
1,me emborraché con 6 club colombia,2021-05-19 23:51:07,Bogotá,"Bogotá,D.C.,Colombia",1395165149653356546,https://twitter.com/i/web/stuatus/139516514965...
2,Carracas a ponerse en el pecho la camisa de mi...,2021-05-19 23:51:06,Bogotá,Bogota,1395165147916914692,https://twitter.com/i/web/stuatus/139516514791...
3,"A pesar de todo, que emoción ser hija de esta ...",2021-05-19 23:51:06,Bogotá,Colombia,1395165144989241345,https://twitter.com/i/web/stuatus/139516514498...
4,"Para los interesados, acá esta en PDF el infor...",2021-05-19 23:51:03,Bogotá,@jhpelaez ™,1395165135312916481,https://twitter.com/i/web/stuatus/139516513531...


In [137]:
import snscrape.modules.twitter as sntwitter

datoslib = []
maxTweets = 100  
for i,tweet in enumerate(sntwitter.TwitterSearchScraper('#AngelinaJolie').get_items()) :
        if i > maxTweets :
            break
        temporal = []
        temporal.append(tweet)
        temporal.append(tweet.date)
        temporal.append(tweet.content)
        datoslib.append(temporal)
        print(tweet)
        print(tweet.date)
        print(tweet.content)
        print("*"*100)
        print()
   


datos = pd.DataFrame(datoslib)
datos.columns = ['tweet', 'date', 'contenido']


https://twitter.com/Rodaje35/status/1395167532055764997
2021-05-20 00:00:35+00:00
Recuerda que ¡Ya está en cines  #ThoseWhoWishMeDead (Aquellos que desean mi muerte) estelarizada por #AngelinaJolie! https://t.co/SoIb3AGxPE
****************************************************************************************************

https://twitter.com/Toastiewiththe/status/1395162492687486976
2021-05-19 23:40:33+00:00
#AngelinaJolie Anji Daly
#WomanCrushWednesday https://t.co/rssWoLmz5F
****************************************************************************************************

https://twitter.com/Toastiewiththe/status/1395161504861786114
2021-05-19 23:36:38+00:00
#AngelinaJolie This little one will be 15 in 8 days!!!😱
Shiloh https://t.co/QQ61YPGEEH
****************************************************************************************************

https://twitter.com/MrLK07/status/1395158851536637952
2021-05-19 23:26:05+00:00
I watched #ThoseWhoWishMeDead and 
I'm sorry, but we NEED 

In [138]:
datos = datos[['tweet', 'date', 'contenido']]
datos.head()

,tweet,date,contenido
0,(https://twitter.com/Rodaje35/status/139516753...,2021-05-20 00:00:35+00:00,Recuerda que ¡Ya está en cines #ThoseWhoWishM...
1,(https://twitter.com/Toastiewiththe/status/139...,2021-05-19 23:40:33+00:00,#AngelinaJolie Anji Daly\n#WomanCrushWednesday...
2,(https://twitter.com/Toastiewiththe/status/139...,2021-05-19 23:36:38+00:00,#AngelinaJolie This little one will be 15 in 8...
3,(https://twitter.com/MrLK07/status/13951588515...,2021-05-19 23:26:05+00:00,"I watched #ThoseWhoWishMeDead and \nI'm sorry,..."
4,(https://twitter.com/Toastiewiththe/status/139...,2021-05-19 23:25:17+00:00,#AngelinaJolie IG joliecharacters\nDo you know...


In [139]:
#limpiar data
import re

pattern1 = '?P<pic>pic.twitter.com/[^\s]+'
pattern2 = '?P<url>https?://[^\s]+'

def text_clean(row):
    text = row['contenido']
    
    links = [tuple(j for j in i if j)[-1] for i in re.findall(f"({pattern1})|({pattern2})",text)]
    for link in links:
        text = text.replace(link,"")
             
    hashtags = [interaction for interaction in text.split() if interaction.startswith("#")]
    for hashtag in hashtags:
        text = text.replace(hashtag,"")
        
    mentions = [interaction for interaction in text.split() if interaction.startswith("@")]
    for mention in mentions:
        text = text.replace(mention,"")
        
    return text, links, hashtags, mentions

In [140]:
datos[['clean', 'links', 'hashtags', 'mentions']] = datos.apply(text_clean, axis=1, result_type='expand')

datos.sample()

,tweet,date,contenido,clean,links,hashtags,mentions
19,(https://twitter.com/WilliamBanksy/status/1395...,2021-05-19 22:03:28+00:00,@chrissyteigen is the weaker half of a #PowerC...,is the weaker half of a,[],"[#PowerCouple, #jayz, #bradpitt, #AngelinaJolie]","[@chrissyteigen, @Beyonce, @kanyewest, @KimKar..."


In [141]:

def get_emojis(text):
    emoji_list = []
    data = regex.findall(r'\X', text)
    for word in data:
        if any(char in emoji.UNICODE_EMOJI['en'] for char in word):
            emoji_list.append(word)

    return emoji_list

datos['emojis'] = datos['clean'].apply(lambda text: get_emojis(text))

In [142]:
data.head()

,tweets,date,city,user_location,id,link
0,#ATENCION | Urgente\n@ArchivaColombia @WOLA_or...,2021-05-19 23:51:08,Bogotá,Holanda,1395165156779479040,https://twitter.com/i/web/stuatus/139516515677...
1,me emborraché con 6 club colombia,2021-05-19 23:51:07,Bogotá,"Bogotá,D.C.,Colombia",1395165149653356546,https://twitter.com/i/web/stuatus/139516514965...
2,Carracas a ponerse en el pecho la camisa de mi...,2021-05-19 23:51:06,Bogotá,Bogota,1395165147916914692,https://twitter.com/i/web/stuatus/139516514791...
3,"A pesar de todo, que emoción ser hija de esta ...",2021-05-19 23:51:06,Bogotá,Colombia,1395165144989241345,https://twitter.com/i/web/stuatus/139516514498...
4,"Para los interesados, acá esta en PDF el infor...",2021-05-19 23:51:03,Bogotá,@jhpelaez ™,1395165135312916481,https://twitter.com/i/web/stuatus/139516513531...


In [144]:
datos['fecha'] = datos.date.dt.date
temp = pd.DataFrame(datos.fecha.value_counts()).reset_index()
temp.columns = ['fecha', 'cnt']
temp = temp.sort_values('fecha')
temp

,fecha,cnt
0,2021-05-19,100
1,2021-05-20,1


In [148]:
trace = go.Scatter(x=temp.fecha.values,
                   y=temp.cnt.values,
                   text=[f"Fecha: {f}<br>Tuits: {c}" for f,c in zip(temp.fecha.values,temp.cnt.values)],
                   hoverinfo='text',
                   mode='lines+markers',
                   name='Días',
                   line={'color':'#1DA1F2'})

layout = go.Layout(title='Número de tuits por día')

fig = go.Figure(data=[trace], layout=layout)
iplot(fig)

In [145]:
from collections import Counter

temp_emojis = pd.DataFrame(list(zip(Counter(sum(datos.emojis.values,[])).keys(),Counter(sum(datos.emojis.values,[])).values())))
temp_emojis.columns = ['emoji', 'cnt']
temp_emojis.sort_values('cnt', ascending=False, inplace=True)
temp_emojis.head()

,emoji,cnt
4,⭐,11
6,😍,8
22,🔥,7
5,⭐️,5
1,👀,5


In [146]:
trace = go.Bar(x=temp_emojis.emoji.values[:15],
               y=temp_emojis.cnt.values[:15])

layout = go.Layout(title='Emojis más usados')

fig = go.Figure(data=[trace], layout=layout)
iplot(fig)